In [9]:
import numpy as np
import os
# import PIL
# import PIL.Image
import tensorflow as tf
# import tensorflow_datasets as tfds
import pathlib
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

dataDir = "/code/archive/garbage_classification/Garbage_classification"

imgHeight = 384
imgWidth = 512
batchSize = 32

trainDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

valDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

AUTOTUNE = tf.data.AUTOTUNE

trainDs = trainDs.cache().prefetch(buffer_size=AUTOTUNE)
valDs = valDs.cache().prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(imgSize, imgSize),
  layers.Rescaling(1./255)
])

# Apply preprocessing to images to make them compatible with pretrained network
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=(imgHeight,imgWidth,3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze cnn base network
base_model.trainable = False      

# Preprocessing layer:
# Convert 224x224x3 images to 5x5x1280 using pretrained neural network
image_batch, label_batch = next(iter(trainDs))
feature_batch = base_model(image_batch)

# Get input shape to be processede by a dense neural network
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

prediction_layer = tf.keras.layers.Dense(5)
prediction_batch = prediction_layer(feature_batch_average)

# Build neural network architecture
inputs = tf.keras.Input(shape=(imgHeight,imgWidth,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs,outputs)


# Compile model
base_learning_rate = 0.0001
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model.fit(trainDs,
epochs=10,
validation_data=valDs)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


Found 2390 files belonging to 5 classes.
Using 1912 files for training.
Found 2390 files belonging to 5 classes.
Using 478 files for validation.


2022-12-04 21:20:50.841803: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Epoch 1/20
60/60 [==============================] - 118s 2s/step - loss: 1.0454 - accuracy: 0.5999 - val_loss: 0.7109 - val_accuracy: 0.7678
Epoch 2/20
60/60 [==============================] - 109s 2s/step - loss: 0.6297 - accuracy: 0.7819 - val_loss: 0.5642 - val_accuracy: 0.8013
Epoch 3/20
 8/60 [===>..........................] - ETA: 1:18 - loss: 0.5592 - accuracy: 0.8086

KeyboardInterrupt: 